In [210]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import re
from datetime import datetime

# Initiate the connectoin to the database

In [211]:
# Connect to MongoDB
server_adress = 'mesiin592022-0031.westeurope.cloudapp.azure.com:30000' 
client = MongoClient(server_adress)

# Select database
db = client['dblp']

# Select collection
collection = db['awards']

# Queries

In [212]:
queries = []

queries.append([
    {
        '$group': {
            '_id': {
                '$toLower': '$institution.city_name'
            }, 
            'montant_cumul': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'montant_cumul': -1
        }
    }, {
        '$project': {
            'City Name': '$_id', 
            'Cumulative Amount': '$montant_cumul', 
            '_id': 0
        }
    }
])

In [213]:
queries.append([
    {
        '$match': {
            'institution.city_name': re.compile(r"New York(?i)")
        }
    }, {
        '$group': {
            '_id': {
                'instName': '$institution.name', 
                'instZip': '$institution.zipcode'
            }, 
            'quantity': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'quantity': -1
        }
    }, {
        '$project': {
            'Insitution name': '$_id.instName', 
            'Insitution zip': '$_id.instZip', 
            'Awards received': '$quantity', 
            '_id': 0
        }
    }
])

In [214]:
queries.append([
    {
        '$match': {
            'investigators.email_id': {
                '$ne': None
            }
        }
    }, {
        '$group': {
            '_id': '$investigators.email_id', 
            'setDomaines': {
                '$addToSet': '$foa_info'
            }, 
            'email': {
                '$first': '$investigators.email_id'
            }
        }
    }, {
        '$project': {
            'email': '$_id', 
            'Number of domains': {
                '$size': '$setDomaines'
            }, 
            '_id': 0
        }
    }, {
        '$sort': {
            'Number of domains': -1
        }
    }, {
        '$limit': 10
    }
])

In [215]:
queries.append([
    {
        '$match': {
            'institution.state_name': 'California'
        }
    }, {
        '$group': {
            '_id': '$organisation_code', 
            'nbInstitutions': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'nbInstitutions': -1
        }
    }, {
        '$limit': 5
    }
])

In [216]:
queries.append([
    {
        '$group': {
            '_id': '$institution', 
            'count': {
                '$sum': 1
            }, 
            'averageAwardAmount': {
                '$avg': '$amount'
            }, 
            'maxAwardAmount': {
                '$max': '$amount'
            }, 
            'minAwardAmount': {
                '$min': '$amount'
            }, 
            'stdAwardAmount': {
                '$stdDevPop': '$amount'
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }
])

In [217]:
queries.append([
    {
        '$unwind': {
            'path': '$programs', 
            'includeArrayIndex': 'string', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$group': {
            '_id': '$programs', 
            'nb_amount': {
                '$sum': '$amount'
            }
        }
    }, {
        '$sort': {
            'nb_amount': -1
        }
    }, {
        '$project': {
            '_id': 1, 
            'nb_amount': 1
        }
    }
])

In [218]:
queries.append([
    {
        '$match': {
            'investigators.email_id': re.compile(r"(a)(?i)")
        }
    }, {
        '$group': {
            '_id': {
                'date': {
                    '$substr': [
                        '$effective_date', 6, -1
                    ]
                }, 
                'invest': '$investigators.email_id'
            }, 
            'nb': {
                '$sum': 1
            }
        }
    }, {
        '$project': {
            'Investigator': '$_id.invest', 
            'Date': '$_id.date', 
            'Number': '$nb', 
            '_id': 0
        }
    }
])

In [219]:
queries.append([
    {
        '$match': {
            'investigators.email_id': 'jeremygtaylor@compuserve.com'
        }
    }, {
        '$project': {
            'Email': '$investigators.email_id', 
            'date': {
                '$toDate': '$investigators.start_date'
            }, 
            'today': datetime.utcnow()
        }
    }, {
        '$addFields': {
            '_id': '$_id', 
            'dateDiff': {
                '$subtract': [
                    '$today', '$date'
                ]
            }
        }
    }, {
        '$project': {
            'Email': 1, 
            'dateDiff': 1, 
            'secs': {
                '$divide': [
                    '$dateDiff', 1000
                ]
            }
        }
    }, {
        '$project': {
            'Email': 1, 
            'mins': {
                '$divide': [
                    '$secs', 60
                ]
            }
        }
    }, {
        '$project': {
            'Email': 1, 
            'hours': {
                '$divide': [
                    '$mins', 60
                ]
            }
        }
    }, {
        '$project': {
            'Email': 1, 
            'days': {
                '$divide': [
                    '$hours', 24
                ]
            }
        }
    }, {
        '$group': {
            '_id': '$Email', 
            'avgDays': {
                '$avg': '$days'
            }
        }
    }, {
        '$project': {
            'Email': '$_id', 
            'Average days': '$avgDays', 
            '_id': 0
        }
    }
])

# Definitions

In [220]:
def get_aggregation_execTimeMs(query, db, collection):
    cursor = db.command(
        'explain', 
        {
            'aggregate': "awards", 
            'pipeline': query, 
            'cursor': {}
        }, 
        verbosity='executionStats'
    )
    times = []
    # Keep only the longest execution time
    for shard in cursor["shards"].keys():
        times.append(cursor["shards"][shard]["stages"][0]["$cursor"]["executionStats"]["executionTimeMillis"])
    return np.max(times)

In [221]:
def run_query_n_times(query, db, collection, n):
    times = []
    for i in range(n):
        t = get_aggregation_execTimeMs(query, db, collection)
        print(f"Ran in {t}ms")
        times.append(t)
    return times

In [222]:
def initialize_df(queries, n):
    times_list = [f"{i}_stats" for i in range(1, n+1)] # List of stats columns
    stats_list = ['min_runtime', 'max_runtime', 'all_avg_runtime']
    columns = ['query_number', 'query'] + times_list + stats_list
    
    df = pd.DataFrame(np.zeros((len(queries),len(columns))), columns=columns) # Create dataframe
    
    df['query_number'] = [i for i in range(1, len(queries)+1)]
    df['query'] = queries

    return df, times_list, stats_list

# Initialize a dataframe

In [223]:
n = 10 # Number of times to run each query
df, times_list, stats_list = initialize_df(queries, n)
df

,query_number,query,1_stats,2_stats,3_stats,4_stats,5_stats,6_stats,7_stats,8_stats,9_stats,10_stats,min_runtime,max_runtime,all_avg_runtime
0,1,[{'$group': {'_id': {'$toLower': '$institution...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,[{'$match': {'institution.city_name': re.compi...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,[{'$match': {'investigators.email_id': {'$ne':...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,[{'$match': {'institution.state_name': 'Califo...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,"[{'$group': {'_id': '$institution', 'count': {...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,"[{'$unwind': {'path': '$programs', 'includeArr...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,[{'$match': {'investigators.email_id': re.comp...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,[{'$match': {'investigators.email_id': 'jeremy...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Loop through the queries

In [224]:
num_shards = 2
for query in queries:
    query_number = queries.index(query) + 1
    print(f"Running query {query_number}")
    times = run_query_n_times(query, db, "awards", n)
    
    print("Times for query", query_number, ":", times)
    df.loc[df['query_number'] == query_number, times_list] = times
    df.loc[df['query_number'] == query_number, 'min_runtime'] = min(times)
    df.loc[df['query_number'] == query_number, 'max_runtime'] = max(times)
    df.loc[df['query_number'] == query_number, 'all_avg_runtime'] = np.mean(times)
    
    # Drop min and max values
    times.remove(min(times))
    times.remove(max(times))
    print("Times without outliers for query", query_number, ":", times)
    
    df.loc[df['query_number'] == query_number, 'avg_runtime_no_outliers'] = np.mean(times)
    
df.to_csv(f"results/S{num_shards}_results.csv", index=False)

Running query 1
Ran in 2937ms
Ran in 2926ms
Ran in 2928ms
Ran in 2922ms
Ran in 2947ms
Ran in 2935ms


In [ ]:
df

,query_number,query,1_stats,2_stats,3_stats,4_stats,min_runtime,max_runtime,all_avg_runtime,avg_runtime_no_outliers
0,1,[{'$group': {'_id': {'$toLower': '$institution...,2960.0,2939.0,2935.0,2926.0,2926.0,2960.0,2940.00,2937.0
1,2,[{'$match': {'institution.city_name': re.compi...,1009.0,1011.0,1010.0,1010.0,1009.0,1011.0,1010.00,1010.0
2,3,[{'$match': {'investigators.email_id': {'$ne':...,1429.0,1432.0,1430.0,1421.0,1421.0,1432.0,1428.00,1429.5
3,4,[{'$match': {'institution.state_name': 'Califo...,1128.0,1126.0,1125.0,1126.0,1125.0,1128.0,1126.25,1126.0
4,5,"[{'$group': {'_id': '$institution', 'count': {...",4467.0,4471.0,4634.0,4544.0,4467.0,4634.0,4529.00,4507.5
5,6,"[{'$unwind': {'path': '$programs', 'includeArr...",2620.0,2569.0,2590.0,2600.0,2569.0,2620.0,2594.75,2595.0
6,7,[{'$match': {'investigators.email_id': re.comp...,1504.0,1502.0,1493.0,1500.0,1493.0,1504.0,1499.75,1501.0
7,8,[{'$match': {'investigators.email_id': 'jeremy...,4.0,3.0,4.0,3.0,3.0,4.0,3.50,3.5


In [ ]:
# import matplotlib.pyplot as plt

# shards_list = [f"S{i}_average_no_outliers" for i in range(1, 7)]

# # Plot the results with a bar chart for each query and for each shard
# fig, ax = plt.subplots(1, 1, figsize=(20, 10))
# df.plot.bar(x='query_number', y=df[shards_list], ax=ax)
# plt.title("Average runtime of each query for each shard")
# plt.xlabel("Query number")
# plt.ylabel("Average runtime (ms)")
# plt.show()